In [1]:
# Aufarbeitung der NAN Werte

In [2]:
from pathlib import Path
from scipy import stats

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

np.set_printoptions(suppress=True)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

from sklearn.linear_model import LogisticRegression

from IPython.display import display, Markdown

In [3]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [4]:
path1 = Path(r"A:\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")
path2 = Path(r"C:\Users\rober\Documents\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")

if path1.is_dir():
    DATASET_DIR = path1
else:
    DATASET_DIR = path2

In [5]:
app_train = pd.read_csv(DATASET_DIR / "3. Ausreißerbereinigung" / "app_train.csv")
bureau = pd.read_csv(DATASET_DIR / "3. Ausreißerbereinigung" / "bureau.csv")
pa_pos = pd.read_csv(DATASET_DIR / "3. Ausreißerbereinigung" / "pa_pos.csv")
pa_ip = pd.read_csv(DATASET_DIR / "3. Ausreißerbereinigung" / "pa_ip.csv")
pa_cc = pd.read_csv(DATASET_DIR / "3. Ausreißerbereinigung" / "pa_cc.csv")
description = pd.read_csv(DATASET_DIR / "HomeCredit_columns_description.csv", encoding="latin", index_col=0)

In [6]:
app_train = app_train.set_index("SK_ID_CURR")
bureau = bureau.set_index("SK_ID_CURR")
pa_pos = pa_pos.set_index("SK_ID_CURR")
pa_ip = pa_ip.set_index("SK_ID_CURR")
pa_cc = pa_cc.set_index("SK_ID_CURR")

In [7]:
len(app_train)

307511

In [8]:
app_train.head()

,TARGET,A_CODE_GENDER,A_NAME_INCOME_TYPE,A_NAME_EDUCATION_TYPE,A_OCCUPATION_TYPE,A_REGION_RATING_CLIENT,A_REG_CITY_NOT_WORK_CITY,A_FLAG_DOCUMENT_3,A_AMT_INCOME_TOTAL,A_AMT_CREDIT,A_REGION_POPULATION_RELATIVE,A_DAYS_BIRTH,A_DAYS_EMPLOYED,A_DAYS_REGISTRATION,A_DAYS_ID_PUBLISH,A_EXT_SOURCE_1,A_EXT_SOURCE_2,A_EXT_SOURCE_3,A_BASEMENTAREA_AVG,A_YEARS_BEGINEXPLUATATION_AVG,A_LANDAREA_AVG,A_NONLIVINGAREA_AVG,A_TOTALAREA_MODE,A_DAYS_LAST_PHONE_CHANGE,A_CREDIT/INCOME
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,
100002,0,0,0,0,0,0,0,0,202500.0,406597.5,0.018801,-9461,-637,-3648.0,-2120,0.083037,0.262949,0.139376,0.0369,0.9722,0.0369,0.0000,0.0149,-1134.0,2.007889
100003,1,1,1,1,1,1,0,0,270000.0,1293502.5,0.003541,-16765,-1188,-1186.0,-291,0.311267,0.622246,NaN,0.0529,0.9851,0.0130,0.0098,0.0714,-828.0,4.790750
100004,1,0,0,0,0,0,0,1,67500.0,135000.0,0.010032,-19046,-225,-4260.0,-2531,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,-815.0,2.000000
100006,1,1,0,0,0,0,0,0,135000.0,312682.5,0.008019,-19005,-3039,-9833.0,-2437,NaN,0.650442,NaN,NaN,NaN,NaN,NaN,NaN,-617.0,2.316167
100007,1,0,0,0,1,0,1,1,121500.0,513000.0,0.028663,-19932,-3038,-4311.0,-3458,NaN,0.322738,NaN,NaN,NaN,NaN,NaN,NaN,-1106.0,4.222222


In [9]:
result = pd.DataFrame(index=app_train.index)
result.index.name = "SK_ID_CURR"

In [10]:
result["application"] = True

result["bureau"] = False
result.loc[bureau.index.values, "bureau"] = True

result["pa_pos"] = False
result.loc[pa_pos.index.values, "pa_pos"] = True

result["pa_ip"] = False
result.loc[pa_ip.index.values, "pa_ip"] = True

result["pa_cc"] = False
result.loc[pa_cc.index.values, "pa_cc"] = True

In [11]:
# mögliche Kombinationen
combi = result.drop_duplicates()
combi = combi.sort_values(list(combi.columns.values), ascending=(False,False,False,False,False))
combi = combi.reset_index()
combi = combi.drop(["SK_ID_CURR"], axis=1)
combi

,application,bureau,pa_pos,pa_ip,pa_cc
0,True,True,True,True,True
1,True,True,True,True,False
2,True,True,True,False,True
3,True,True,True,False,False
4,True,True,False,True,True
5,True,True,False,True,False
6,True,True,False,False,True
7,True,True,False,False,False
8,True,False,True,True,True
9,True,False,True,True,False


In [12]:
T = []
for idx, row in combi.iterrows():
    A = app_train.index.values
    if row["bureau"] == True:
        A = np.intersect1d(A, bureau.index.values)
    if row["pa_pos"] == True:
        A = np.intersect1d(A, pa_pos.index.values)
    if row["pa_ip"] == True:
        A = np.intersect1d(A, pa_ip.index.values)
    if row["pa_cc"] == True:
        A = np.intersect1d(A, pa_cc.index.values)
    T.append(len(A))

In [13]:
combi["CNT"] = T

In [14]:
combi["PER"] = combi["CNT"] / len(app_train) * 100

In [15]:
# Schwäche im Datensatz -> Beschreiben

In [16]:
combi.sort_values("PER")

,application,bureau,pa_pos,pa_ip,pa_cc,CNT,PER
0,True,True,True,True,True,57402,18.666649
4,True,True,False,True,True,59223,19.258823
2,True,True,True,False,True,65001,21.137780
8,True,False,True,True,True,66702,21.690931
6,True,True,False,False,True,67203,21.853852
12,True,False,False,True,True,68962,22.425864
10,True,False,True,False,True,75223,24.461889
14,True,False,False,False,True,77934,25.343484
1,True,True,True,True,False,150033,48.789474
5,True,True,False,True,False,151865,49.385225


In [17]:
# Zusammenfügen aller Daten

In [18]:
result = pd.DataFrame(index=app_train.index)
result.index.name = "SK_ID_CURR"

In [19]:
result = pd.merge(result, app_train, how="left", left_index=True, right_index=True)
result = pd.merge(result, bureau, how="left", left_index=True, right_index=True)
result = pd.merge(result, pa_pos, how="left", left_index=True, right_index=True)
result = pd.merge(result, pa_ip, how="left", left_index=True, right_index=True)
result = pd.merge(result, pa_cc, how="left", left_index=True, right_index=True)

In [20]:
result.head()

,TARGET,A_CODE_GENDER,A_NAME_INCOME_TYPE,A_NAME_EDUCATION_TYPE,A_OCCUPATION_TYPE,A_REGION_RATING_CLIENT,A_REG_CITY_NOT_WORK_CITY,A_FLAG_DOCUMENT_3,A_AMT_INCOME_TOTAL,A_AMT_CREDIT,A_REGION_POPULATION_RELATIVE,A_DAYS_BIRTH,A_DAYS_EMPLOYED,A_DAYS_REGISTRATION,A_DAYS_ID_PUBLISH,A_EXT_SOURCE_1,A_EXT_SOURCE_2,A_EXT_SOURCE_3,A_BASEMENTAREA_AVG,A_YEARS_BEGINEXPLUATATION_AVG,A_LANDAREA_AVG,A_NONLIVINGAREA_AVG,A_TOTALAREA_MODE,A_DAYS_LAST_PHONE_CHANGE,A_CREDIT/INCOME,B_Active,B_Closed,B_CREDIT_DAY_OVERDUE,B_DAYS_CREDIT_ENDDATE,B_AMT_CREDIT_SUM,B_AMT_CREDIT_SUM_DEBT,B_AMT_CREDIT_SUM_OVERDUE,B_AMT_ANNUITY,PA_CNT_CREDITS_x,PA_AMT_ANNUITY_x,PA_AMT_APPLICATION_x,PA_AMT_CREDIT_x,PA_AMT_DOWN_PAYMENT_x,PA_AMT_GOODS_PRICE_x,PA_RATE_DOWN_PAYMENT_x,PA_POS_CNT_PAYMENTS_LEFT,PA_POS_CNT_DPD,PA_CNT_NAME_CONTRACT_TYPE_Revolving loans_x,PA_CNT_NAME_CONTRACT_STATUS_Refused_x,PA_CNT_CODE_REJECT_REASON_HC_x,PA_CNT_NAME_CLIENT_TYPE_New_x,PA_CNT_NAME_CLIENT_TYPE_Refreshed_x,PA_CNT_NAME_PORTFOLIO_Cards_x,PA_CNT_NAME_PRODUCT_TYPE_walk-in_x,PA_CNT_NAME_YIELD_GROUP_high_x,PA_CNT_NAME_YIELD_GROUP_low_normal_x,PA_CNT_PRODUCT_COMBINATION_Card Street_x,PA_CNT_PRODUCT_COMBINATION_Cash X-Sell: low_x,PA_CNT_PRODUCT_COMBINATION_POS industry with interest_x,PA_CNT_CREDITS_y,PA_AMT_ANNUITY_y,PA_AMT_APPLICATION_y,PA_AMT_CREDIT_y,PA_AMT_DOWN_PAYMENT_y,PA_AMT_GOODS_PRICE_y,PA_RATE_DOWN_PAYMENT_y,PA_IP_PAYMENT_30,PA_IP_DAYS_DIFF_30,PA_IP_AMT_DIFF_30,PA_IP_PAYMENT_60,PA_IP_DAYS_DIFF_60,PA_IP_AMT_DIFF_60,PA_IP_PAYMENT_90,PA_IP_DAYS_DIFF_90,PA_IP_AMT_DIFF_90,PA_IP_PAYMENT_120,PA_IP_DAYS_DIFF_120,PA_IP_AMT_DIFF_120,PA_IP_PAYMENT_150,PA_IP_DAYS_DIFF_150,PA_IP_AMT_DIFF_150,PA_IP_PAYMENT_180,PA_IP_DAYS_DIFF_180,PA_IP_AMT_DIFF_180,PA_CNT_NAME_CONTRACT_TYPE_Revolving loans_y,PA_CNT_NAME_CONTRACT_STATUS_Refused_y,PA_CNT_CODE_REJECT_REASON_HC_y,PA_CNT_NAME_CLIENT_TYPE_New_y,PA_CNT_NAME_CLIENT_TYPE_Refreshed_y,PA_CNT_NAME_PORTFOLIO_Cards_y,PA_CNT_NAME_PRODUCT_TYPE_walk-in_y,PA_CNT_NAME_YIELD_GROUP_high_y,PA_CNT_NAME_YIELD_GROUP_low_normal_y,PA_CNT_PRODUCT_COMBINATION_Card Street_y,PA_CNT_PRODUCT_COMBINATION_Cash X-Sell: low_y,PA_CNT_PRODUCT_COMBINATION_POS industry with interest_y,PA_CNT_CREDITS,PA_AMT_ANNUITY,PA_AMT_APPLICATION,PA_AMT_CREDIT,PA_AMT_GOODS_PRICE,PA_CC_BALANCE_30,PA_CC_PAYMENT_30,PA_CC_DPD_30,PA_CC_BALANCE_60,PA_CC_PAYMENT_60,PA_CC_DPD_60,PA_CC_BALANCE_90,PA_CC_PAYMENT_90,PA_CC_DPD_90,PA_CC_BALANCE_120,PA_CC_PAYMENT_120,PA_CC_DPD_120,PA_CC_BALANCE_150,PA_CC_PAYMENT_150,PA_CC_DPD_150,PA_CC_BALANCE_180,PA_CC_PAYMENT_180,PA_CC_DPD_180,PA_CNT_NAME_CONTRACT_TYPE_Revolving loans,PA_CNT_NAME_CONTRACT_STATUS_Refused,PA_CNT_CODE_REJECT_REASON_HC,PA_CNT_NAME_CLIENT_TYPE_New,PA_CNT_NAME_CLIENT_TYPE_Refreshed,PA_CNT_NAME_PORTFOLIO_Cards,PA_CNT_NAME_PRODUCT_TYPE_walk-in,PA_CNT_NAME_YIELD_GROUP_high,PA_CNT_NAME_YIELD_GROUP_low_normal,PA_CNT_PRODUCT_COMBINATION_Card Street,PA_CNT_PRODUCT_COMBINATION_Cash X-Sell: low,PA_CNT_PRODUCT_COMBINATION_POS industry with interest
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,0,0,0,0,0,0,0,0,202500.0,406597.5,0.018801,-9461,-637,-3648.0,-2120,0.083037,0.262949,0.139376,0.0369,0.9722,0.0369,0.0000,0.0149,-1134.0,2.007889,2.0,6.0,0.0,309.0,212745.0,81927.0,0.0,0.0,1.0,9251.775,179055.00,179055.0,0.00,179055.00,0.000000,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,9251.775,179055.0,179055.0,0.0,179055.0,0.000000,2.041241,-24.0,0.0,-0.408248,-12.0,0.0,-0.408248,-14.0,0.0,-0.408248,-18.0,0.0,-0.408248,-15.0,0.0,-0.408248,-19.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,1,1,1,1,1,1,0,0,270000.0,1293502.5,0.003541,-16765,-1188,-1186.0,-291,0.311267,0.622246,NaN,0.0529,0.9851,0.0130,0.0098,0.0714,-828.0,4.790750,1.0,3.0,0.0,1216.0,810000.0,0.0,0.0,0.0,3.0,56553.990,435436.50,484191.0,2295.00,4

In [ ]:
x = result.drop(["TARGET"], axis=1)
imp = IterativeImputer(max_iter=500, random_state=0)
df = imp.fit_transform(x)

In [ ]:
df.head()

In [ ]:
df.to_csv(DATASET_DIR / "2. Datenaufbereitung" / "datensatz.csv")